# First of all, you must load the extension

Alternatively, you can add the loading logic to a startup python script for ipython. Read [this](https://ipython.org/ipython-doc/1/config/overview.html#startup-files)

In [1]:
%load_ext fugue_notebook

# The simplest case, running using NativeExecutionEngine

In [2]:
%%fsql
CREATE [[0]] SCHEMA a:int
PRINT

,a
0,0


# Test %%fsql parsing

In [3]:
from fugue_notebook.env import FugueSQLMagics
from fugue import register_execution_engine, register_default_execution_engine
from fugue import NativeExecutionEngine
from pytest import raises


class MockEngine(NativeExecutionEngine):
    def __init__(self, conf=None):
        super().__init__(conf=conf)
        
def test_fugue_sql_magic():
    register_execution_engine("m", lambda conf, **kwargs: MockEngine(conf=conf))
    m = FugueSQLMagics(None, {"a": 1}, {"b": 2})
    e = m.get_engine("   ", {})
    assert isinstance(e, NativeExecutionEngine)
    assert 1 == e.conf["a"]
    assert 2 == e.conf["b"]

    e = m.get_engine(' {"a":2,"c":3, "b":2}  ', {})
    assert isinstance(e, NativeExecutionEngine)
    assert 2 == e.conf["a"]
    assert 2 == e.conf["b"]
    assert 3 == e.conf["c"]

    with raises(ValueError):
        e = m.get_engine(' {"a":2,"b":1}  ', {})

    e = m.get_engine(" m   ", {})
    assert isinstance(e, MockEngine)
    assert 1 == e.conf["a"]
    assert 2 == e.conf["b"]

    e = m.get_engine(' m  {"a":2,"c":3, "b":2}  ', {})
    assert isinstance(e, MockEngine)
    assert 2 == e.conf["a"]
    assert 2 == e.conf["b"]
    assert 3 == e.conf["c"]

test_fugue_sql_magic()

# Run with a different execution engine

This engine is defined in previous test

In [4]:
%%fsql m
CREATE [[0]] SCHEMA a:int
PRINT

,a
0,0


# Communicate between fsql cells

You must use `yield` to communicate betweem cells. That means all the intermediate data will be stored in file system. So you should also specify `fugue.workflow.checkpoint.path`.

In [9]:
%%fsql native
a = CREATE [[0]] SCHEMA a:int YIELD DATAFRAME

In [10]:
%%fsql native
SELECT * FROM a
PRINT

,a
0,0
